In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data,batch_size=64)
test_dataloader = DataLoader(test_data,batch_size=64)

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten=nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10),
    )

  def forward(self,x):
    x=self.flatten(x)
    logits=self.linear_relu_stack(x)
    return logits

model = NeuralNetwork()

## Hyperparameters
We define the following hyperparameters for training:


*   **Number of Epochs**: the number times to iterate over the dataset
*   **Batch Size**: the number of data samples propagated through the network before the parameters are updated
*   **Learning Rate**: how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training.



In [6]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop
Once we set our hyperparameter, we can then train and optimize our model with an optimization loop. Each iteration of the optimization loop is called an **epoch**.
Each epoch consists of two main parts:


*   **The Train Loop**-iterate over the training dataset and try to converge to optimal parameters
*   **The Validation/Test Loop**-iterate over the test dataset to check if model performance is improving.



## Loss Function
When presented with some training data, our untrained network is likely not to give the correct answer. Loss function measures the degree of dissimilarity of obtained result to the target value, and it is the **loss function** that we want to minimize during training. To calculate the loss we make a prediction using the inputs of our given data sample and compare it against the true data label value.

Common loss functions include `nn.MSELoss` (Mean Square Error) for regression tasks, and `nn.NLLLoss` (Negative Log Likelihood) for classification. `nn.CrossEntropyLoss` combines nn.LogSoftmax and nn.NLLLoss.

We pass our model’s output logits to `nn.CrossEntropyLoss`, which will normalize the logits and compute the prediction error.

In [7]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

## Optimizer
Optimization is the process of adjusting model parameters to reduce model error in each training step. **Optimization algorithms** define how this process is performed (in this example we use Stochastic Gradient Descent). All optimization logic is encapsulated in the optimizer object. Here, we use the SGD optimizer; additionally, there are many different optimizers available in PyTorch such as ADAM and RMSProp, that work better for different kinds of models and data.

We initialize the optimizer by registering the model’s parameters that need to be trained, and passing in the learning rate hyperparameter.

In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Inside the training loop, optimization happens in three steps:
Call `optimizer.zero_grad()` to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.
Backpropagate the prediction loss with a call to `loss.backward()`. PyTorch deposits the gradients of the loss w.r.t. each parameter.
Once we have our gradients, we call `optimizer.step()` to adjust the parameters by the gradients collected in the backward pass.


## Full Implementation

In [10]:
def train_loop(dataloader,model,loss_fn,optimizer):
  size = len(dataloader.dataset)

  model.train()
  for batch,(X,y) in enumerate(dataloader):
    # Computer prediction and loss
    pred = model(X)
    loss = loss_fn(pred,y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch_size%100==0:
      loss,current = loss.item(),(batch + 1) * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 45.9%, Avg loss: 2.163295 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 57.5%, Avg loss: 1.900530 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 61.2%, Avg loss: 1.526779 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 63.0%, Avg loss: 1.251021 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 64.2%, Avg loss: 1.084155 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 65.3%, Avg loss: 0.979243 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 66.7%, Avg loss: 0.908517 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 67.8%, Avg loss: 0.857735 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 69.0%, Avg loss: 0.819261 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 70.2%, Avg loss: 0.788673 

Done!
